# Chapter 56: Multi‑Variate Time‑Series

## Learning Objectives

By the end of this chapter, you will be able to:

- Understand the need for multi‑variate time‑series models when multiple interrelated variables are observed over time
- Apply classical multi‑variate models like Vector Autoregression (VAR) to capture linear dependencies among series
- Implement deep learning approaches such as multivariate LSTMs and sequence‑to‑sequence models for forecasting multiple related time series
- Use graph neural networks to model explicit relationships between variables (e.g., stocks in the same sector)
- Incorporate attention mechanisms to focus on relevant series and time steps
- Understand dynamic factor models for dimensionality reduction in high‑dimensional settings
- Perform causal analysis using Granger causality tests to discover lead‑lag relationships
- Evaluate multi‑variate forecasts using metrics that account for cross‑series correlations
- Apply these methods to the NEPSE system, where multiple stocks interact through market dynamics

---

## Introduction

So far, we have often treated each stock in the NEPSE as an independent time series, building separate models for each symbol or using the same model with symbol as a feature. However, in financial markets, stocks do not move in isolation. They are influenced by common factors (e.g., market index, interest rates, sector news) and by each other (e.g., correlated movements, lead‑lag relationships). Ignoring these inter‑dependencies can lead to suboptimal forecasts and missed opportunities.

**Multi‑variate time‑series** modeling explicitly accounts for the relationships among multiple time series. By modeling them jointly, we can:

- Improve forecast accuracy by leveraging information from related series.
- Understand the dynamic interactions (e.g., how a shock to one stock propagates to others).
- Generate consistent forecasts across a portfolio (e.g., ensuring predicted prices for different stocks are plausible together).

In this chapter, we will explore a range of multi‑variate techniques, from classical econometric models to modern deep learning architectures. Using the NEPSE dataset, we will model a basket of stocks simultaneously, capturing the rich interdependencies of the Nepalese market.

---

## 56.1 Why Multi‑Variate Time‑Series?

Consider predicting the closing price of NABIL (Nepal Bank Ltd.). Its price is influenced not only by its own history but also by:

- The performance of other banks (sector effect).
- The overall market index (NEPSE index).
- Macroeconomic indicators (interest rates, remittance inflows).
- News or events affecting related industries.

A univariate model for NABIL can only use its own lags, missing these external drivers. A multi‑variate model can include other stocks, the index, and external variables as inputs, potentially improving predictions.

Moreover, for portfolio management, we need joint forecasts of multiple assets to estimate correlations and diversification benefits. Multi‑variate models provide not only point forecasts but also predictive distributions that capture cross‑series covariances.

In the NEPSE context, we might model the top 10 most liquid stocks together, or model each stock alongside the sector index and a market index.

---

## 56.2 Vector Autoregression (VAR)

The Vector Autoregression (VAR) model is the workhorse of classical multi‑variate time‑series analysis. It models each variable as a linear function of its own lagged values and the lagged values of all other variables in the system.

A VAR of order `p` (VAR(p)) is defined as:

`y_t = c + A_1 y_{t-1} + A_2 y_{t-2} + ... + A_p y_{t-p} + ε_t`

where `y_t` is a vector of `k` time series at time `t`, `c` is a vector of intercepts, `A_i` are `k × k` coefficient matrices, and `ε_t` is a vector of white noise innovations (typically with covariance matrix `Σ`).

VAR models are estimated equation‑by‑equation using ordinary least squares (OLS), as each equation has the same regressors (all lagged variables). The main challenge is choosing the lag order `p`, often via information criteria (AIC, BIC) or cross‑validation.

### 56.2.1 Implementing VAR with statsmodels

We'll use the `statsmodels` library to fit a VAR model to NEPSE returns for a few stocks.

```python
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

# Load daily returns for a selection of stocks
# Assume we have a DataFrame with columns: Date, NABIL, NTC, SBI, HRL, NICA (returns)
df = pd.read_csv('nepse_returns.csv', index_col='Date', parse_dates=True)

# Check stationarity (returns should be stationary)
for col in df.columns:
    result = adfuller(df[col].dropna())
    print(f"{col}: p-value = {result[1]:.4f}")

# Fit VAR model
model = VAR(df)
# Select lag order using AIC
results_aic = model.select_order(maxlags=15)
print(results_aic.summary())

# Fit with chosen lag (e.g., p=5)
p = results_aic.selected_orders['aic']
var_model = model.fit(p)
print(var_model.summary())

# Forecast next 5 days
lag_order = var_model.k_ar
forecast = var_model.forecast(df.values[-lag_order:], steps=5)
forecast_df = pd.DataFrame(forecast, index=pd.date_range(start=df.index[-1], periods=5+1, freq='D')[1:],
                           columns=df.columns)
print(forecast_df)
```

**Explanation:**  
We first ensure the series are stationary (returns usually are). The `select_order` method computes information criteria for different lags. We pick the lag that minimizes AIC. The `VAR` object is then fit with that lag. The `forecast` method generates multi‑step forecasts using the estimated coefficients. These forecasts are conditional on the last observed values.

**Interpreting VAR coefficients:**  
Each equation gives the effect of lagged variables on the target. For example, the coefficient of `NABIL(t-1)` in the equation for `NTC(t)` measures how much a 1% return in NABIL yesterday affects NTC's return today, holding other variables constant. Granger causality tests (see later) can formally test these relationships.

### 56.2.2 Impulse Response Functions

A key output of VAR models is the **impulse response function (IRF)**, which traces the effect of a one‑time shock to one variable on the entire system over time.

```python
# Compute impulse responses
irf = var_model.irf(periods=10)
irf.plot(orth=False)  # orthogonalized vs non‑orthogonalized
plt.show()
```

**Explanation:**  
The IRF shows, for example, how a shock to NABIL's return propagates to other stocks in subsequent days. Orthogonalized IRFs use a Cholesky decomposition of the residual covariance matrix to isolate shocks, which depends on the ordering of variables.

### 56.2.3 Limitations of VAR

- Linear only; cannot capture non‑linear dependencies.
- Parameter explosion: with `k` variables and lag `p`, we estimate `k + p*k²` parameters. For large `k`, this becomes infeasible (curse of dimensionality).
- Assumes stationarity and constant relationships over time.

For NEPSE with many stocks, VAR may overfit. We can reduce dimensionality by including only a few key stocks or using factor models.

---

## 56.3 Multivariate LSTM

Long Short‑Term Memory (LSTM) networks can naturally handle multi‑variate inputs by having the input layer accept a vector of features at each time step. For multi‑variate forecasting, we typically use a **sequence‑to‑sequence** architecture: input a window of past observations for all variables, output predictions for one or more future steps for one or all variables.

### 56.3.1 Multi‑variate Input, Multi‑variate Output

We want to predict future values of all `k` variables given their joint history. This can be done with a single LSTM layer followed by a dense layer with `k` units.

**Data preparation:** We create sequences of length `window` as inputs, and the next value (or next `horizon` values) as targets.

```python
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

def create_sequences(data, window, horizon=1):
    """
    data: numpy array of shape (n_samples, n_features)
    returns X (n_sequences, window, n_features), y (n_sequences, n_features)
    """
    X, y = [], []
    for i in range(len(data) - window - horizon + 1):
        X.append(data[i:i+window])
        y.append(data[i+window:i+window+horizon].mean(axis=0))  # average over horizon
    return np.array(X), np.array(y)

# Assume df is a DataFrame of returns for multiple stocks
data = df.values.astype(np.float32)
window = 10
X, y = create_sequences(data, window)

# Train/val/test split (chronological)
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

# Convert to tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.float32)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset = TensorDataset(X_val_t, y_val_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)  # important: no shuffle for time series
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
```

Now define an LSTM model:

```python
class MultiVariateLSTM(nn.Module):
    def __init__(self, n_features, hidden_size, num_layers, horizon=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_features, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_features * horizon)  # predict all features for horizon steps

    def forward(self, x):
        # x shape: (batch, seq_len, n_features)
        out, _ = self.lstm(x)  # out: (batch, seq_len, hidden_size)
        out = out[:, -1, :]     # last time step
        out = self.fc(out)      # (batch, n_features * horizon)
        return out

model = MultiVariateLSTM(n_features=data.shape[1], hidden_size=64, num_layers=2, horizon=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            output = model(batch_X)
            loss = criterion(output, batch_y)
            val_loss += loss.item()
    print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.6f}, Val Loss: {val_loss/len(val_loader):.6f}")
```

**Explanation:**  
We create sequences of past observations. The LSTM processes the sequence and the last hidden state is passed to a fully connected layer that outputs predictions for all variables (for the next time step). This model captures temporal dependencies within each series and cross‑series dependencies because the LSTM sees all series together.

### 56.3.2 Sequence‑to‑Sequence for Multi‑step Forecasting

For multi‑step ahead forecasts (e.g., predict next 5 days), we can modify the output layer to produce `horizon × n_features` values. Alternatively, we can use an encoder‑decoder architecture where the decoder produces one step at a time, feeding its output back as input. This is more complex but can be more flexible.

**Simpler approach:** Use a direct multi‑output model as above, training to predict the average over the horizon, or to predict each step separately with a larger output layer.

---

## 56.4 Graph Neural Networks for Time‑Series

Graph Neural Networks (GNNs) have emerged as a powerful tool for modeling data with explicit relational structure. In finance, we often have a natural graph: stocks are nodes, and edges can represent industry sector, correlation, or supply chain relationships. GNNs allow information to propagate along these edges, capturing how one stock's movement affects another.

### 56.4.1 Defining the Graph

For NEPSE, we might define edges based on:

- **Sector membership**: Stocks in the same sector (banks, hydropower, insurance) are connected.
- **Correlation threshold**: Connect stocks whose historical returns are highly correlated.
- **Lead‑lag relationships**: Directed edges based on Granger causality.

We need an adjacency matrix `A` where `A_ij` indicates the strength of connection from `j` to `i` (often symmetric and normalized).

**Example: Building a correlation‑based graph**

```python
import networkx as nx
import pandas as pd

# Compute correlation matrix of returns
corr = df.corr().values
# Threshold to create adjacency matrix (e.g., |corr| > 0.5)
adj = np.where(np.abs(corr) > 0.5, 1, 0)
np.fill_diagonal(adj, 0)  # no self‑loops

# Convert to a graph
G = nx.from_numpy_array(adj)
```

### 56.4.2 Graph Convolutional LSTM

We can combine a GNN with an LSTM by replacing the linear transformations in the LSTM with graph convolutions. The **Graph Convolutional LSTM (GC‑LSTM)** cell uses graph convolution operations on the input and hidden state.

For simplicity, we'll use the `torch_geometric` library to implement a model that processes node features over time.

**Example: Using a simple GNN + LSTM**

We'll treat each time step as a graph with node features (price, volume, technical indicators). We first apply a GNN at each time step to update node features based on neighbors, then feed the sequence of updated features to an LSTM.

```python
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader as GeoDataLoader

class GNN_LSTM(nn.Module):
    def __init__(self, node_features, hidden_size, num_nodes, num_layers):
        super().__init__()
        self.num_nodes = num_nodes
        self.gcn = GCNConv(node_features, hidden_size)
        self.lstm = nn.LSTM(input_size=hidden_size * num_nodes,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True)
        self.fc = nn.Linear(hidden_size, num_nodes)  # predict next value for each node

    def forward(self, x_seq, edge_index):
        # x_seq: (batch, seq_len, num_nodes, node_features)
        batch_size, seq_len, _, _ = x_seq.shape
        # Process each time step with GCN
        out_seq = []
        for t in range(seq_len):
            # x_t: (batch, num_nodes, node_features)
            x_t = x_seq[:, t, :, :].reshape(batch_size * self.num_nodes, -1)
            # Apply GCN
            x_t = self.gcn(x_t, edge_index)
            x_t = F.relu(x_t)
            x_t = x_t.view(batch_size, -1)  # flatten nodes
            out_seq.append(x_t)
        # Stack along sequence
        lstm_in = torch.stack(out_seq, dim=1)  # (batch, seq_len, hidden_size * num_nodes)
        lstm_out, _ = self.lstm(lstm_in)       # (batch, seq_len, hidden_size)
        last_out = lstm_out[:, -1, :]          # last hidden
        pred = self.fc(last_out)               # (batch, num_nodes)
        return pred
```

**Explanation:**  
At each time step, we run a graph convolution on node features, using a fixed adjacency `edge_index`. The GCN aggregates neighbor information. The resulting node embeddings are flattened and fed to an LSTM that captures temporal dynamics. Finally, a linear layer produces predictions for each node (stock).

**Training:** We need to create a batched version where each sample is a sequence of graphs. This is complex but manageable with careful reshaping.

---

## 56.5 Attention Mechanisms for Multivariate Forecasting

Attention mechanisms, especially the Transformer architecture, have become popular for multivariate time‑series. They can capture long‑range dependencies and allow the model to focus on relevant parts of the input sequence and relevant variables.

### 56.5.1 Transformer for Multivariate Time‑Series

A Transformer encoder can process the sequence of multivariate observations. We add positional encoding to retain order, and the self‑attention layers allow each time step to attend to all others, capturing cross‑time and cross‑variable interactions implicitly.

**Example with PyTorch's TransformerEncoder**

```python
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)  # (1, max_len, d_model)

    def forward(self, x):
        # x: (batch, seq_len, d_model)
        return x + self.pe[:, :x.size(1)]

class TransformerTimeSeries(nn.Module):
    def __init__(self, n_features, d_model=64, nhead=4, num_layers=3, horizon=1):
        super().__init__()
        self.input_proj = nn.Linear(n_features, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(d_model, n_features * horizon)

    def forward(self, x):
        # x: (batch, seq_len, n_features)
        x = self.input_proj(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = x[:, -1, :]  # take last time step's representation
        out = self.output_proj(x)
        return out

model = TransformerTimeSeries(n_features=data.shape[1], d_model=64, nhead=4, num_layers=3, horizon=1)
```

**Explanation:**  
The input sequence is projected to `d_model` dimensions, positional encoding added, then passed through Transformer encoder layers. The output at the last position is used for prediction. The self‑attention mechanism can learn which past time steps and which variables are most relevant.

### 56.5.2 Interpretability with Attention Weights

One advantage of attention is that we can inspect the attention weights to understand what the model focuses on. For multivariate forecasting, we can see which stocks and time lags are most influential for a given prediction.

```python
# To get attention weights, we need to modify the forward to return them
# This is model‑specific; here we show a simple way to extract from a single layer
def forward_with_attention(self, x):
    x = self.input_proj(x)
    x = self.pos_encoder(x)
    # Get attention from the first layer
    attn_weights = []
    for layer in self.transformer_encoder.layers:
        x, attn = layer(x, return_attention=True)  # requires custom layer modification
        attn_weights.append(attn)
    x = x[:, -1, :]
    out = self.output_proj(x)
    return out, attn_weights
```

**Note:** Standard PyTorch Transformer layers do not return attention by default; you may need to use a custom implementation or a library like `HuggingFace`'s `Bert` adapted for time‑series.

---

## 56.6 Dynamic Factor Models

When the number of series `k` is large (e.g., all NEPSE stocks), a VAR would have too many parameters. **Dynamic Factor Models (DFMs)** reduce dimensionality by assuming that a small number of latent factors drive the co‑movements of the series. Each observed series is a linear combination of the common factors plus an idiosyncratic component.

`y_t = Λ f_t + ε_t`

where `f_t` is a vector of `r` latent factors (r << k), `Λ` is a factor loading matrix, and `ε_t` is idiosyncratic noise. The factors themselves often follow a VAR process: `f_t = Φ f_{t-1} + η_t`.

DFMs can be estimated via principal components (for the static part) and state‑space methods (Kalman filter) for the dynamics.

### 56.6.1 Estimating Factors with PCA

A simple two‑step approach: estimate factors by PCA on the covariance matrix of `y_t`, then model the factors with a VAR.

```python
from sklearn.decomposition import PCA

# Assume Y is (n_samples, n_series)
pca = PCA(n_components=5)  # choose number of factors
factors = pca.fit_transform(Y)  # (n_samples, 5)

# Now treat factors as a multivariate time series and fit a VAR
model_factors = VAR(factors)
results = model_factors.fit(ic='aic')
```

**Explanation:**  
PCA extracts the top principal components, which are linear combinations of the original series that explain most variance. These are treated as factors. Then we model the dynamics of the factors with a VAR. To forecast a specific series, we predict the factors and multiply by the corresponding loadings.

### 56.6.2 State‑Space Formulation and Kalman Filter

For a more rigorous treatment, we can set up a state‑space model and estimate using the Kalman filter. The `statsmodels` library has a `DynamicFactorMQ` class for medium‑to‑large dimensions.

```python
from statsmodels.tsa.statespace.dynamic_factor_mq import DynamicFactorMQ

# Fit a dynamic factor model with 2 factors
dfm = DynamicFactorMQ(Y, factors=2, factor_orders=1)  # factor VAR order 1
dfm_result = dfm.fit()

# Forecast
forecast = dfm_result.forecast(steps=5)
```

**Explanation:**  
`DynamicFactorMQ` handles missing data and can incorporate multiple blocks of series (e.g., sectors). It estimates factors and loadings via maximum likelihood and can produce forecasts.

---

## 56.7 Causal Inference and Granger Causality

Understanding which variables *cause* others is crucial for building interpretable models and for policy decisions (e.g., if we can influence one variable, does it affect others?). **Granger causality** is a statistical concept: a variable `X` Granger‑causes `Y` if past values of `X` help predict `Y` beyond past values of `Y` alone.

In a VAR framework, we can test Granger causality using F‑tests or chi‑squared tests.

### 56.7.1 Granger Causality Test with statsmodels

```python
from statsmodels.tsa.stattools import grangercausalitytests

# Test whether NABIL returns Granger‑cause NTC returns
data_pair = df[['NABIL', 'NTC']].dropna()
gc_result = grangercausalitytests(data_pair, maxlag=5, verbose=True)
```

**Explanation:**  
For each lag, the test reports F‑statistic and p‑value. A low p‑value (e.g., <0.05) indicates that NABIL helps predict NTC. Note that Granger causality is about predictive ability, not true causality (which would require controlled experiments).

For multiple variables, we can test all pairs and build a directed graph. This graph can then be used as the adjacency matrix for a GNN.

---

## 56.8 Feature Interactions and Cross‑Correlation

Even without a full model, we can explore cross‑correlations at various lags to understand lead‑lag relationships. The **cross‑correlation function (CCF)** plots correlation between two series at different lags.

```python
import statsmodels.tsa.api as smt

# Compute CCF between NABIL and NTC
ccf = smt.ccf(df['NABIL'], df['NTC'], adjusted=False)
lags = range(-10, 11)
plt.stem(lags, ccf[:21])  # first 21 lags (including negative)
plt.xlabel('Lag')
plt.ylabel('Cross‑correlation')
plt.title('Cross‑correlation: NABIL vs NTC')
plt.show()
```

**Explanation:**  
Positive lags mean NABIL leads NTC (NABIL at t‑k correlates with NTC at t). Negative lags mean NTC leads NABIL. This can inform which lags to include in a multivariate model.

---

## 56.9 Implementation Considerations for NEPSE

When applying multivariate methods to NEPSE data, consider:

- **Choice of stocks**: Too many stocks will overburden the model. Start with a subset (e.g., top 10 by liquidity) or use factor models.
- **Data frequency**: Daily data is common. For higher frequency, computational demands increase.
- **Non‑stationarity**: Use returns or log‑returns, not prices.
- **Exogenous variables**: Include market index, sector indices, or macro variables if available.
- **Validation**: Use walk‑forward validation (time‑based splits) to evaluate forecasting performance.

**Example pipeline for NEPSE multivariate forecasting:**

```python
# 1. Select a set of stocks (e.g., 5 from different sectors)
symbols = ['NABIL', 'NTC', 'HRL', 'SBI', 'NICA']
df_returns = get_returns(symbols)  # DataFrame with columns as symbols, index date

# 2. Fit a VAR model (or LSTM, etc.)
# 3. Forecast next day returns for all symbols
# 4. Evaluate using portfolio metrics (e.g., mean squared error per symbol, or portfolio return if acted upon)
```

---

## 56.10 Evaluation of Multivariate Models

Evaluating multivariate forecasts is more complex than univariate because we care about both per‑series accuracy and the joint distribution (e.g., correlations). Common metrics:

- **Per‑series RMSE/MAE**: Still useful to see if each series is well predicted.
- **Average RMSE across series**: `mean(rmse_i)`.
- **Trace of covariance matrix of errors**: Measures total squared error across series.
- **Log‑likelihood**: For probabilistic forecasts, the joint log‑likelihood under a multivariate normal assumption.
- **Portfolio performance**: If forecasts are used to construct a portfolio, evaluate the portfolio's return and risk.

**Example: Computing average RMSE**

```python
from sklearn.metrics import mean_squared_error

# y_test_true: (n_test, n_series)
# y_test_pred: (n_test, n_series)
rmse_per_series = np.sqrt(mean_squared_error(y_test_true, y_test_pred, multioutput='raw_values'))
avg_rmse = np.mean(rmse_per_series)
print(f"RMSE per series: {rmse_per_series}")
print(f"Average RMSE: {avg_rmse:.4f}")
```

**Example: Evaluating correlation of errors**

```python
errors = y_test_true - y_test_pred
corr_errors = np.corrcoef(errors.T)  # (n_series, n_series)
# We want the errors to be uncorrelated ideally, so off‑diagonal close to zero
mean_abs_corr = np.mean(np.abs(corr_errors[np.triu_indices_from(corr_errors, k=1)]))
print(f"Mean absolute error correlation: {mean_abs_corr:.4f}")
```

A good model should have low error correlation, meaning it captures the dependencies well.

---

## Chapter Summary

In this chapter, we explored the rich field of multi‑variate time‑series modeling and its application to the NEPSE stock market. We covered:

- The motivation for modeling multiple interrelated series jointly.
- Classical Vector Autoregression (VAR), including lag selection, impulse responses, and Granger causality.
- Deep learning approaches: multivariate LSTM and sequence‑to‑sequence models.
- Graph Neural Networks to leverage explicit relationships among stocks.
- Attention mechanisms and Transformers for capturing complex dependencies.
- Dynamic Factor Models for dimensionality reduction when many series exist.
- Causal analysis with Granger causality tests.
- Cross‑correlation analysis to discover lead‑lag relationships.
- Practical considerations and evaluation metrics for multivariate forecasting.

By modeling multiple stocks together, we can achieve better forecasts and gain insights into the market structure. For the NEPSE system, multivariate models can improve portfolio‑level predictions and risk assessments. In the next chapter, we will discuss **Hierarchical and Grouped Time‑Series**, which deals with time series that have aggregation constraints (e.g., regional sales, or sector indices and their constituents).

---

**End of Chapter 56**